# Assignment 1

Deadline: 19.03.2025, 12:00 CET

<Add your name, student-id and emal address>

In [7]:
# Import standard libraries
import os
import sys
import timeit # To compute runtimes
from typing import Optional

# Import third-party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import local modules
project_root = '/home/crow/Documents/UZH/QPM/qpmwp-course'   # Change this path if needed
src_path = os.path.join(project_root, 'src')
sys.path.append(src_path)
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.optimization import Optimization, Objective
from optimization.optimization_data import OptimizationData
from optimization.quadratic_program import QuadraticProgram, USABLE_SOLVERS

## 1. Solver horse race

### 1.a)
(3 points)

Generate a Multivariate-Normal random dataset of dimension TxN, T=1000, N=100, and compute a vector of expected returns, q, and a covariance matrix, P, using classes ExpectedReturn and Covariance respectively.

In [8]:

# Set the dimensions
T = 1000 # Number of time periods
N = 100 # Number of assets

# Generate a random mean vector and covariance matrix for the multivariate normal distribution
mean = np.random.uniform(0, 0.1, N)
cov = np.random.uniform(0, 0.1, (N, N))
cov = np.dot(cov, cov.T) # Ensure the covariance matrix is positive definite

# Generate the Multivariate-Normal random dataset
data = np.random.multivariate_normal(mean, cov, T)

# Convert the dataset to a DataFrame for easier manipulation
df = pd.DataFrame(data, columns=[f'Asset_{i+1}' for i in range(N)])

# Compute the vector of expected returns (mean returns) from df
q = df.mean().values

# Compute the covariance matrix from df
P = P = df.cov().values

# Display the results
print("Vector of expected returns (q):")
print(q)

print("\nCovariance matrix (P):")
print(P)

Vector of expected returns (q):
[ 0.01284069  0.09178656  0.07542201  0.05374297  0.06363197  0.07843118
  0.10433525  0.01334098  0.03322056  0.06230795  0.06972748  0.02611011
  0.06947807  0.0625738   0.07136209  0.1008782   0.02441639  0.03024021
  0.01960806  0.05174424  0.05888857  0.01317618  0.0518258   0.01081571
  0.01031869 -0.02168022  0.04744793  0.08496465  0.00319791  0.0680608
  0.02277653  0.03458859  0.06374689  0.08581085  0.07984443  0.07466015
  0.04381303  0.0392261   0.028536    0.01336974  0.08833257  0.08764137
  0.08454818  0.07952174  0.02978259  0.02292482  0.05539312  0.07620713
  0.01023465  0.03914359  0.05918776  0.06427914  0.05444334  0.05094463
  0.0822268   0.07585606  0.05774919  0.04234771  0.02249608  0.10032753
  0.03361965  0.08993499  0.01081305  0.05377679  0.04436474  0.0358514
  0.02998965  0.07670784  0.0252667   0.06806367  0.04336953  0.07523668
  0.01551997  0.07664304  0.03246875  0.06830802  0.02393347  0.02346691
  0.07385536  0.06218

### 1.b)
(3 points)

Instantiate a constraints object by injecting column names of the data created in 1.a) as ids and add:
- a budget constaint (i.e., asset weights have to sum to one)
- lower bounds of 0.0 for all assets
- upper bounds of 0.2 for all assets
- group contraints such that the sum of the weights of the first 30 assets is <= 0.3, the sum of assets 31 to 60 is <= 0.4 and the sum of assets 61 to 100 is <= 0.5

In [ ]:
# Instantiate the Constraints class
constraints = Constraints(ids = df.columns.tolist())

# Add budget constraint
#<your code here>

# Add box constraints (i.e., lower and upper bounds)
#<your code here>

# Add linear constraints
#<your code here>

### 1.c) 
(4 points)

Solve a Mean-Variance optimization problem (using coefficients P and q in the objective function) which satisfies the above defined constraints.
Repeat the task for all open-source solvers in qpsolvers and compare the results in terms of:

- runtime
- accuracy: value of the primal problem.
- reliability: are all constarints fulfilled? Extract primal resisduals, dual residuals and duality gap.

Generate a DataFrame with the solvers as column names and the following row index: 'solution_found': bool, 'objective': float, 'primal_residual': float, 'dual_residual': float, 'duality_gap': float, 'runtime': float.

Put NA's for solvers that failed for some reason (e.g., unable to install the package or solvers throws an error during execution). 




In [ ]:
# Extract the constraints in the format required by the solver
GhAb = constraints.to_GhAb()

# Loop over solvers, instantiate the quadratic program, solve it and store the results
#<your code here>

Print and visualize the results

In [5]:
#<your code here>

## 2. Analytical Solution to Minimum-Variance Problem

(5 points)

- Create a `MinVariance` class that follows the structure of the `MeanVariance` class.
- Implement the `solve` method in `MinVariance` such that if `solver_name = 'analytical'`, the analytical solution is computed and stored within the object (if such a solution exists). If not, call the `solve` method from the parent class.
- Create a `Constraints` object by injecting the same ids as in part 1.b) and add a budget constraint.
- Instantiate a `MinVariance` object by setting `solver_name = 'analytical'` and passing instances of `Constraints` and `Covariance` as arguments.
- Create an `OptimizationData` object that contains an element `return_series`, which consists of the synthetic data generated in part 1.a).
- Solve the optimization problem using the created `MinVariance` object and compare the results to those obtained in part 1.c).


In [ ]:
# Define class MinVariance
class MinVariance(Optimization):

    def __init__(self,
                 constraints: Constraints,
                 covariance: Optional[Covariance] = None,
                 **kwargs):
        super().__init__(
            constraints=constraints,
            **kwargs
        )
        self.covariance = Covariance() if covariance is None else covariance

    def set_objective(self, optimization_data: OptimizationData) -> None:
        #<your code here>

    def solve(self) -> None:
        if self.params.get('solver_name') == 'analytical':
            #<your code here>
            # check for inequlity contraints
            return None
        else:
            return super().solve()


# Create a constraints object with just a budget constraint
#<your code here>

# Instantiate the MinVariance class
#<your code here>

# Prepare the optimization data and prepare the optimization problem
#<your code here>

# Solve the optimization problem and print the weights
#<your code here>